### Finance – Ensuring Accurate Transactions

**Task 1**: Transaction Data Validation Insights

**Objective**: Maintain transaction integrity.

**Steps**:
1. Choose a sample financial transaction dataset.
2. Identify common transaction issues like duplicate entries or incorrect amounts.
3. Develop a list of validation checks specific to financial transactions.

In [1]:
# Write your code from here
import pandas as pd
from datetime import datetime

# Sample transaction dataset
data = {
    'Transaction_ID': ['TXN001', 'TXN002', 'TXN002', 'TXN004', 'TXN005', None],
    'Date': ['2025-05-01', '2025-05-01', '2025-05-01', '2025-06-20', '2025-05-02', '2025-05-03'],
    'Account_ID': ['AC1234', 'AC1234', 'AC1234', 'AC5678', 'AC5678', 'AC9999'],
    'Amount': [150.00, 150.00, 150.00, -25.00, 20000.00, 0.00],
    'Currency': ['USD', 'USD', 'USD', 'USD', 'USD', 'EUR'],
    'Transaction_Type': ['DEBIT', 'DEBIT', 'DEBIT', 'CREDIT', 'DEBIT', 'DEBIT'],
    'Merchant': ['Amazon', 'Amazon', 'Amazon', 'Refund', 'Tesla', ''],
    'Status': ['Completed', 'Completed', 'Completed', 'Completed', 'Completed', 'Completed']
}

df = pd.DataFrame(data)

# Convert Date to datetime
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Today's date for reference
today = pd.Timestamp(datetime.today().date())

# --- Validation Checks ---

# 1. Duplicate Transactions
duplicates = df[df.duplicated(subset=['Account_ID', 'Amount', 'Date'], keep=False)]

# 2. Negative Amounts for DEBIT
invalid_debits = df[(df['Transaction_Type'] == 'DEBIT') & (df['Amount'] < 0)]

# 3. Amount Out of Range
amount_out_of_range = df[(df['Amount'] < 0) | (df['Amount'] > 10000)]

# 4. Missing Required Fields
missing_required = df[df[['Transaction_ID', 'Amount', 'Date', 'Status']].isnull().any(axis=1)]

# 5. Future Dates
future_dates = df[df['Date'] > today]

# 6. Inconsistent Transaction Type & Amount
type_amount_mismatch = df[
    ((df['Transaction_Type'] == 'DEBIT') & (df['Amount'] < 0)) |
    ((df['Transaction_Type'] == 'CREDIT') & (df['Amount'] > 0))
]

# 7. Invalid Currencies
valid_currencies = ['USD', 'EUR', 'GBP']
invalid_currency = df[~df['Currency'].isin(valid_currencies)]

# 8. Invalid Transaction Status
valid_status = ['Completed', 'Pending', 'Failed']
invalid_status = df[~df['Status'].isin(valid_status)]

# 9. Duplicate Transaction IDs
duplicate_ids = df[df.duplicated(subset=['Transaction_ID'], keep=False)]

# --- Print Results ---
print("Duplicate Transactions:\n", duplicates, "\n")
print("Invalid DEBIT Transactions:\n", invalid_debits, "\n")
print("Amounts Out of Range:\n", amount_out_of_range, "\n")
print("Missing Required Fields:\n", missing_required, "\n")
print("Future-dated Transactions:\n", future_dates, "\n")
print("Type-Amount Mismatches:\n", type_amount_mismatch, "\n")
print("Invalid Currency Codes:\n", invalid_currency, "\n")
print("Invalid Transaction Status:\n", invalid_status, "\n")
print("Duplicate Transaction IDs:\n", duplicate_ids, "\n")


Duplicate Transactions:
   Transaction_ID       Date Account_ID  Amount Currency Transaction_Type  \
0         TXN001 2025-05-01     AC1234   150.0      USD            DEBIT   
1         TXN002 2025-05-01     AC1234   150.0      USD            DEBIT   
2         TXN002 2025-05-01     AC1234   150.0      USD            DEBIT   

  Merchant     Status  
0   Amazon  Completed  
1   Amazon  Completed  
2   Amazon  Completed   

Invalid DEBIT Transactions:
 Empty DataFrame
Columns: [Transaction_ID, Date, Account_ID, Amount, Currency, Transaction_Type, Merchant, Status]
Index: [] 

Amounts Out of Range:
   Transaction_ID       Date Account_ID   Amount Currency Transaction_Type  \
3         TXN004 2025-06-20     AC5678    -25.0      USD           CREDIT   
4         TXN005 2025-05-02     AC5678  20000.0      USD            DEBIT   

  Merchant     Status  
3   Refund  Completed  
4    Tesla  Completed   

Missing Required Fields:
   Transaction_ID       Date Account_ID  Amount Currency Transa

**Task 2**: Implement Financial Data Validation

**Objective**: Use automated tools to ensure transaction accuracy.

**Steps**:
1. Integrate data validation rules into your existing financial systems.
2. Ensure real-time checks to validate data upon entry.

In [3]:
import pandas as pd
from datetime import datetime
from flask import Flask, request, jsonify
import threading

# -----------------------------
# BATCH VALIDATION FUNCTION
# -----------------------------

def validate_transactions(df):
    today = pd.Timestamp(datetime.today().date())
    required_cols = ['Transaction_ID', 'Amount', 'Date', 'Status']

    results = {
        'duplicates': df[df.duplicated(subset=['Account_ID', 'Amount', 'Date'], keep=False)],
        'invalid_debit': df[(df['Transaction_Type'] == 'DEBIT') & (df['Amount'] < 0)],
        'out_of_range': df[(df['Amount'] < 0) | (df['Amount'] > 10000)],
        'missing_fields': df[df[required_cols].isnull().any(axis=1)],
        'future_dates': df[pd.to_datetime(df['Date'], errors='coerce') > today],
        'invalid_status': df[~df['Status'].isin(['Completed', 'Pending', 'Failed'])],
        'type_mismatch': df[
            ((df['Transaction_Type'] == 'DEBIT') & (df['Amount'] < 0)) |
            ((df['Transaction_Type'] == 'CREDIT') & (df['Amount'] > 0))
        ],
        'invalid_currency': df[~df['Currency'].isin(['USD', 'EUR', 'GBP'])],
        'duplicate_ids': df[df.duplicated(subset=['Transaction_ID'], keep=False)],
    }

    return results

# -----------------------------
# SAMPLE DATA & BATCH VALIDATION RUN
# -----------------------------

def run_batch_validation():
    sample_data = {
        'Transaction_ID': ['TXN001', 'TXN002', 'TXN002', 'TXN004', 'TXN005', None],
        'Date': ['2025-05-01', '2025-05-01', '2025-05-01', '2025-06-20', '2025-05-02', '2026-01-01'],
        'Account_ID': ['AC1234', 'AC1234', 'AC1234', 'AC5678', 'AC5678', 'AC9999'],
        'Amount': [150.00, 150.00, 150.00, -25.00, 20000.00, 0.00],
        'Currency': ['USD', 'USD', 'USD', 'USD', 'USD', 'XYZ'],
        'Transaction_Type': ['DEBIT', 'DEBIT', 'DEBIT', 'CREDIT', 'DEBIT', 'DEBIT'],
        'Merchant': ['Amazon', 'Amazon', 'Amazon', 'Refund', 'Tesla', ''],
        'Status': ['Completed', 'Completed', 'Completed', 'Completed', 'Completed', 'Failed']
    }

    df = pd.DataFrame(sample_data)
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

    print("\n--- Running Batch Validation ---\n")
    results = validate_transactions(df)

    for rule, issues in results.items():
        if not issues.empty:
            print(f"❌ Validation Failed: {rule}")
            print(issues.to_string(index=False), "\n")
        else:
            print(f"✅ Passed: {rule}")

# -----------------------------
# REAL-TIME VALIDATION API
# -----------------------------

app = Flask(__name__)

@app.route('/validate', methods=['POST'])
def validate_transaction():
    txn = request.json
    errors = []
    today = datetime.today().date()

    # Required fields
    for field in ['Transaction_ID', 'Amount', 'Date', 'Status']:
        if field not in txn or txn[field] in [None, ""]:
            errors.append(f"{field} is missing.")

    # Type/amount validation
    if txn.get('Transaction_Type') == 'DEBIT' and txn['Amount'] < 0:
        errors.append("DEBIT amount cannot be negative.")
    if txn.get('Transaction_Type') == 'CREDIT' and txn['Amount'] > 0:
        errors.append("CREDIT amount must be negative.")

    # Currency and status validation
    if txn.get('Currency') not in ['USD', 'EUR', 'GBP']:
        errors.append("Unsupported currency.")
    if txn.get('Status') not in ['Completed', 'Pending', 'Failed']:
        errors.append("Invalid status.")

    # Date validation
    try:
        txn_date = datetime.strptime(txn['Date'], "%Y-%m-%d").date()
        if txn_date > today:
            errors.append("Transaction date cannot be in the future.")
    except:
        errors.append("Invalid date format. Use YYYY-MM-DD.")

    return jsonify({'valid': not errors, 'errors': errors})

# -----------------------------
# RUN BOTH BATCH + API
# -----------------------------

if __name__ == '__main__':
    # Run batch validation in a separate thread
    batch_thread = threading.Thread(target=run_batch_validation)
    batch_thread.start()

    # Run real-time validation API
    print("\nStarting Flask API on http://localhost:5000/validate")
    app.run(debug=False)



Starting Flask API on http://localhost:5000/validate
 * Serving Flask app '__main__'

--- Running Batch Validation ---

❌ Validation Failed: duplicates
Transaction_ID       Date Account_ID  Amount Currency Transaction_Type Merchant    Status
        TXN001 2025-05-01     AC1234   150.0      USD            DEBIT   Amazon Completed
        TXN002 2025-05-01     AC1234   150.0      USD            DEBIT   Amazon Completed
        TXN002 2025-05-01     AC1234   150.0      USD            DEBIT   Amazon Completed 

✅ Passed: invalid_debit
❌ Validation Failed: out_of_range
Transaction_ID       Date Account_ID  Amount Currency Transaction_Type Merchant    Status
        TXN004 2025-06-20     AC5678   -25.0      USD           CREDIT   Refund Completed
        TXN005 2025-05-02     AC5678 20000.0      USD            DEBIT    Tesla Completed 

❌ Validation Failed: missing_fields
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


Transaction_ID       Date Account_ID  Amount Currency Transaction_Type Merchant Status
          None 2026-01-01     AC9999     0.0      XYZ            DEBIT          Failed 

❌ Validation Failed: future_dates
Transaction_ID       Date Account_ID  Amount Currency Transaction_Type Merchant    Status
        TXN004 2025-06-20     AC5678   -25.0      USD           CREDIT   Refund Completed
          None 2026-01-01     AC9999     0.0      XYZ            DEBIT             Failed 

✅ Passed: invalid_status
✅ Passed: type_mismatch
❌ Validation Failed: invalid_currency
Transaction_ID       Date Account_ID  Amount Currency Transaction_Type Merchant Status
          None 2026-01-01     AC9999     0.0      XYZ            DEBIT          Failed 

❌ Validation Failed: duplicate_ids
Transaction_ID       Date Account_ID  Amount Currency Transaction_Type Merchant    Status
        TXN002 2025-05-01     AC1234   150.0      USD            DEBIT   Amazon Completed
        TXN002 2025-05-01     AC1234   1